In [1]:
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
from Unet3D import *
from metrics3D import *
from Datagens import *

from tensorflow import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers.experimental import preprocessing

np.set_printoptions(precision=3, suppress=True)

CSVLogger.on_test_begin = CSVLogger.on_train_begin
CSVLogger.on_test_batch_end = CSVLogger.on_epoch_end
CSVLogger.on_test_end = CSVLogger.on_train_end

In [2]:
model = build_3Dunet(n_channels=4, ker_init='he_normal', dropout=0.2)
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 4)]                                                           
                                                                                                  
 conv3d (Conv3D)                (None, 128, 128, 12  1744        ['input_1[0][0]']                
                                8, 16)                                                            
                                                                                                  
 conv3d_1 (Conv3D)              (None, 128, 128, 12  6928        ['conv3d[0][0]']                 
                                8, 16)                                                        

                                                                                                  
 conv3d_16 (Conv3D)             (None, 64, 64, 64,   16416       ['up_sampling3d_2[0][0]']        
                                32)                                                               
                                                                                                  
 concatenate_2 (Concatenate)    (None, 64, 64, 64,   0           ['conv3d_3[0][0]',               
                                64)                               'conv3d_16[0][0]']              
                                                                                                  
 conv3d_17 (Conv3D)             (None, 64, 64, 64,   55328       ['concatenate_2[0][0]']          
                                32)                                                               
                                                                                                  
 conv3d_18

In [3]:
directory = [f.path for f in os.scandir(PATH) if f.is_dir()]


def generate_ids(dirLst):
    ids = []
    for i in range(0,len(dirLst)):
        ids.append(dirLst[i][dirLst[i].rfind('/')+1:])
    return ids

train_and_test_ids = generate_ids(directory); 

    
train_test_ids, val_ids = train_test_split(train_and_test_ids, train_size=236, test_size=60, random_state=14) 
train_ids, test_ids = train_test_split(train_test_ids, test_size=36, random_state=14) 

In [4]:
print(len(train_ids))
print(len(val_ids))
print(len(test_ids))

200
60
36


In [5]:
training_generator = DataGenerator3D(train_ids)
valid_generator = DataGenerator3D(val_ids)
test_generator = DataGenerator3D(test_ids)

In [6]:
csv_logger = CSVLogger('Logs/training/3D_U_net_resized.log', separator=',', append=True)

eval_csv_logger = CSVLogger('Logs/testing/3D_U_net_resized.log', separator=',', append=True)

callbacks = [
      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1),
        csv_logger
    ]

In [7]:
K.clear_session()

history =  model.fit(training_generator,
                    epochs=100,
                    steps_per_epoch=len(train_ids),
                    callbacks= callbacks,
                    validation_data = valid_generator
                    )  
model.save("Models/3D/3D_U_net_resized.h5")

Epoch 1/100
200/200 [==============================] - 103s 463ms/step - loss: 0.0971 - accuracy: 0.9859 - mean_io_u: 0.6899 - dice_coef: 0.2569 - dice_coef_necrotic: 0.0546 - dice_coef_edema: 0.0466 - dice_coef_enhancing: 0.0146 - val_loss: 0.0472 - val_accuracy: 0.9899 - val_mean_io_u: 0.8022 - val_dice_coef: 0.2663 - val_dice_coef_necrotic: 0.0879 - val_dice_coef_edema: 0.0383 - val_dice_coef_enhancing: 0.0169 - lr: 0.0010
Epoch 2/100
200/200 [==============================] - 92s 459ms/step - loss: 0.0717 - accuracy: 0.9861 - mean_io_u: 0.7463 - dice_coef: 0.2588 - dice_coef_necrotic: 0.0490 - dice_coef_edema: 0.0528 - dice_coef_enhancing: 0.0228 - val_loss: 0.0478 - val_accuracy: 0.9899 - val_mean_io_u: 0.8568 - val_dice_coef: 0.2715 - val_dice_coef_necrotic: 0.0909 - val_dice_coef_edema: 0.1041 - val_dice_coef_enhancing: 0.0258 - lr: 0.0010
Epoch 3/100
200/200 [==============================] - 92s 461ms/step - loss: 0.0463 - accuracy: 0.9908 - mean_io_u: 0.8473 - dice_coef: 0.26

200/200 [==============================] - 93s 463ms/step - loss: 9985780.0000 - accuracy: 0.9386 - mean_io_u: 0.9215 - dice_coef: 0.2476 - dice_coef_necrotic: 0.0114 - dice_coef_edema: 0.0109 - dice_coef_enhancing: 1.6956e-04 - val_loss: 10760501.0000 - val_accuracy: 0.9448 - val_mean_io_u: 0.9290 - val_dice_coef: 0.2480 - val_dice_coef_necrotic: 0.0091 - val_dice_coef_edema: 0.0117 - val_dice_coef_enhancing: 7.1871e-05 - lr: 1.0000e-06
Epoch 35/100
200/200 [==============================] - 92s 459ms/step - loss: 10335969.0000 - accuracy: 0.9384 - mean_io_u: 0.9212 - dice_coef: 0.2475 - dice_coef_necrotic: 0.0111 - dice_coef_edema: 0.0109 - dice_coef_enhancing: 1.2698e-04 - val_loss: 11469397.0000 - val_accuracy: 0.9105 - val_mean_io_u: 0.8884 - val_dice_coef: 0.2476 - val_dice_coef_necrotic: 0.0213 - val_dice_coef_edema: 0.0158 - val_dice_coef_enhancing: 3.3772e-04 - lr: 1.0000e-06
Epoch 36/100
200/200 [==============================] - 92s 459ms/step - loss: 10814063.0000 - accurac

Epoch 52/100
200/200 [==============================] - 94s 470ms/step - loss: 17527008.0000 - accuracy: 0.9452 - mean_io_u: 0.9296 - dice_coef: 0.2496 - dice_coef_necrotic: 0.0091 - dice_coef_edema: 0.0177 - dice_coef_enhancing: 1.2626e-04 - val_loss: 18721256.0000 - val_accuracy: 0.9517 - val_mean_io_u: 0.9374 - val_dice_coef: 0.2491 - val_dice_coef_necrotic: 0.0064 - val_dice_coef_edema: 0.0149 - val_dice_coef_enhancing: 1.8327e-04 - lr: 1.0000e-06
Epoch 53/100
200/200 [==============================] - 91s 456ms/step - loss: 17999484.0000 - accuracy: 0.9427 - mean_io_u: 0.9265 - dice_coef: 0.2496 - dice_coef_necrotic: 0.0107 - dice_coef_edema: 0.0173 - dice_coef_enhancing: 1.1198e-04 - val_loss: 20243162.0000 - val_accuracy: 0.9605 - val_mean_io_u: 0.9483 - val_dice_coef: 0.2482 - val_dice_coef_necrotic: 0.0042 - val_dice_coef_edema: 0.0090 - val_dice_coef_enhancing: 5.1159e-05 - lr: 1.0000e-06
Epoch 54/100
200/200 [==============================] - 92s 457ms/step - loss: 18888716.

Epoch 70/100
200/200 [==============================] - 92s 459ms/step - loss: 32543910.0000 - accuracy: 0.9383 - mean_io_u: 0.9211 - dice_coef: 0.2478 - dice_coef_necrotic: 0.0063 - dice_coef_edema: 0.0167 - dice_coef_enhancing: 1.6986e-04 - val_loss: 33163912.0000 - val_accuracy: 0.9331 - val_mean_io_u: 0.9149 - val_dice_coef: 0.2492 - val_dice_coef_necrotic: 0.0075 - val_dice_coef_edema: 0.0239 - val_dice_coef_enhancing: 2.5319e-04 - lr: 1.0000e-06
Epoch 71/100
200/200 [==============================] - 93s 463ms/step - loss: 33016446.0000 - accuracy: 0.9356 - mean_io_u: 0.9180 - dice_coef: 0.2480 - dice_coef_necrotic: 0.0068 - dice_coef_edema: 0.0184 - dice_coef_enhancing: 1.6502e-04 - val_loss: 34825896.0000 - val_accuracy: 0.9489 - val_mean_io_u: 0.9340 - val_dice_coef: 0.2494 - val_dice_coef_necrotic: 0.0045 - val_dice_coef_edema: 0.0194 - val_dice_coef_enhancing: 1.7640e-04 - lr: 1.0000e-06
Epoch 72/100
200/200 [==============================] - 91s 457ms/step - loss: 34365556.

Epoch 88/100
200/200 [==============================] - 92s 459ms/step - loss: 64384936.0000 - accuracy: 0.9358 - mean_io_u: 0.9182 - dice_coef: 0.2462 - dice_coef_necrotic: 0.0075 - dice_coef_edema: 0.0108 - dice_coef_enhancing: 7.0527e-05 - val_loss: 74048480.0000 - val_accuracy: 0.9640 - val_mean_io_u: 0.9526 - val_dice_coef: 0.2470 - val_dice_coef_necrotic: 0.0011 - val_dice_coef_edema: 0.0057 - val_dice_coef_enhancing: 4.3792e-05 - lr: 1.0000e-06
Epoch 89/100
200/200 [==============================] - 92s 462ms/step - loss: 66754908.0000 - accuracy: 0.9350 - mean_io_u: 0.9171 - dice_coef: 0.2462 - dice_coef_necrotic: 0.0074 - dice_coef_edema: 0.0115 - dice_coef_enhancing: 4.9889e-05 - val_loss: 69172152.0000 - val_accuracy: 0.9384 - val_mean_io_u: 0.9212 - val_dice_coef: 0.2462 - val_dice_coef_necrotic: 0.0097 - val_dice_coef_edema: 0.0072 - val_dice_coef_enhancing: 1.7119e-04 - lr: 1.0000e-06
Epoch 90/100
200/200 [==============================] - 92s 461ms/step - loss: 67908544.

In [ ]:
model = keras.models.load_model('Models/3D/U_net_200_flair_t1ce_t2.h5', 
                                   custom_objects={ 'accuracy' : tf.keras.metrics.MeanIoU(num_classes=4),
                                                   "dice_coef": dice_coef,
                                                   "dice_coef_necrotic": dice_coef_necrotic,
                                                   "dice_coef_edema": dice_coef_edema,
                                                   "dice_coef_enhancing": dice_coef_enhancing
                                                  }, compile=True)

In [8]:
model.evaluate(test_generator, callbacks=eval_csv_logger, verbose=1)

36/36 [==============================] - 8s 215ms/step - loss: 135205376.0000 - accuracy: 0.9645 - mean_io_u: 0.9531 - dice_coef: 0.2467 - dice_coef_necrotic: 0.0023 - dice_coef_edema: 0.0030 - dice_coef_enhancing: 0.0000e+00


[135205376.0,
 0.9644883275032043,
 0.9531323909759521,
 0.24666635692119598,
 0.0023182411678135395,
 0.0029606642201542854,
 0.0]

In [9]:
valid_csv_logger = CSVLogger('Logs/validation/3D_U_net_resized.log', separator=',', append=True)
model.evaluate(valid_generator, callbacks=valid_csv_logger, verbose=1)

60/60 [==============================] - 11s 186ms/step - loss: 110991328.0000 - accuracy: 0.9640 - mean_io_u: 0.9527 - dice_coef: 0.2469 - dice_coef_necrotic: 0.0022 - dice_coef_edema: 0.0041 - dice_coef_enhancing: 2.4352e-05


[110991328.0,
 0.9640367031097412,
 0.9527334570884705,
 0.24690128862857819,
 0.0021608450915664434,
 0.004137449897825718,
 2.4352484615519643e-05]

In [11]:
x, y = training_generator.__getitem__(0)

In [12]:
xx, yy = training_generator.__getitem__(1)

In [14]:
x is xx

False